In [89]:
import pandas as pd

In [90]:
df = pd.read_csv('/content/train.csv',on_bad_lines="skip", engine='python')
# df = df.reset_index(drop=True)

In [91]:
print(df.dtypes)

id        object
title     object
author    object
text      object
label     object
dtype: object


In [92]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [93]:
df.shape

(20822, 5)

In [94]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [95]:
## Drop na values
df = df.dropna()

In [96]:
## Total rows after dropping the null values
df.shape

(18285, 5)

In [97]:
# Check for rows where the dependent variable is not numeric
invalid_rows = df[~df['label'].apply(lambda x: str(x).isdigit())]
print("Invalid rows:", invalid_rows)

Invalid rows:                                                       id  \
19781  Мы с Всемирным банком проводим каждые пять лет...   
19782            Хотя финансовые ограничения тоже вредны   

                                                   title  \
19781                               и в Восточной Европе   
19782   они вызывают определённую нервозность у инвес...   

                                                  author  \
19781   теперь в Турции и на Ближнем Востоке. И после...   
19782   о которых Вы сказали. И мне не нравится. А ко...   

                                      text  \
19781   и в том числе там был вопрос о том   
19782     чтобы этого не было. Надо только   

                                                  label  
19781             как люди воспринимают своё положение.  
19782   чтобы это была дорога с двусторонним движением.  


In [98]:
# Filter out non-numeric rows
df = df[df['label'].apply(lambda x: str(x).isdigit())]

# Convert 'dependent_var' to integer type
df['label'] = df['label'].astype(int)

In [99]:
## After removing null values, index got changed. so index is reset again
df = df.reset_index(drop=True)

In [100]:
df.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
7,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
8,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that...",0
9,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0


In [101]:
## Independent features
X = df.drop('label', axis=1)

In [102]:
## Dependent feature
y = df['label']

In [103]:
X.shape

(18283, 4)

In [104]:
y.shape

(18283,)

In [105]:
import tensorflow as tf

In [106]:
tf.__version__

'2.15.0'

In [107]:
from tensorflow.keras.layers import Embedding # to convert word2vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential #  a plain stack of layers where each layer has exactly one input tensor and one output tensor
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [108]:
voc_size = 5000

### One Hot Encoding


In [109]:
messages = X.copy()

In [110]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [111]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [112]:
# messages.reset_index(inplace=True)

## Data Preprocessing

In [113]:
import nltk
import re # regular expression
from nltk.corpus import stopwords

In [114]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [115]:
## Instead of stemming we can use lemmatization also, but its takes more time to process
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [116]:
corpus = []
for i in range (0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [stemmer.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [117]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [118]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[3206, 1721, 2607, 1032, 2842, 2160, 150, 1408, 1468, 2133],
 [175, 3723, 229, 2707, 1580, 205, 591],
 [912, 4489, 4336, 3946],
 [4133, 2841, 114, 2557, 4788, 4406],
 [788, 1580, 198, 4782, 3938, 1085, 1580, 1352, 4802, 3172],
 [1843,
  3712,
  4374,
  2340,
  3971,
  3112,
  3391,
  129,
  278,
  453,
  4531,
  4126,
  4087,
  1507,
  591],
 [1788, 4176, 2695, 989, 3680, 2896, 597, 2573, 3650, 4041, 1796],
 [789, 1242, 3743, 2791, 4308, 691, 3112, 2982, 3650, 4041, 1796],
 [698, 3335, 4483, 4568, 794, 2836, 3870, 3917, 3112, 1077],
 [627, 1627, 4689, 1400, 3517, 3384, 3823, 3182],
 [87, 3795, 1026, 654, 3933, 626, 4214, 4344, 3233, 2437, 2872],
 [2557, 2306, 2842, 2836, 3112, 4308],
 [4948, 2012, 3238, 4747, 2687, 750, 1471, 2174, 299],
 [2075, 736, 1098, 3225, 2782, 4100, 1700, 3650, 4041, 1796],
 [4247, 2608, 3802, 266, 985, 3650, 4041, 1796],
 [1999, 3629, 2145, 1167, 227, 2747, 1660, 4483, 4715, 3060],
 [4451, 545, 3723],
 [1972, 1003, 4506, 2975, 3112, 3566, 2281, 591],
 [1931, 

In [119]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [120]:
onehot_repr[1]

[175, 3723, 229, 2707, 1580, 205, 591]

## Embedding Representation

In [121]:
## Inorder convert words to vectors all the sentence should be in same length. So assigning some fixed length and applying padding technique.
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 1408, 1468, 2133],
       [   0,    0,    0, ..., 1580,  205,  591],
       [   0,    0,    0, ..., 4489, 4336, 3946],
       ...,
       [   0,    0,    0, ..., 3650, 4041, 1796],
       [   0,    0,    0, ..., 3916, 4959, 3428],
       [   0,    0,    0, ..., 3328, 1905, 3503]], dtype=int32)

In [122]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  175, 3723,  229, 2707, 1580,  205,  591], dtype=int32)

In [123]:
## Creating model: applying embedding layer and LSTM
feature_dimension = 40
model = Sequential()
model.add(Embedding(voc_size, feature_dimension, input_length = sent_length))
model.add(LSTM(100))  # 100 neuron
model.add(Dense(1, activation='sigmoid')) # o/p is binary (0s and 1s) so sigmoid activation function
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [124]:
model.summary() # paranthese indicates the method call.

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [125]:
len(embedded_docs), y.shape

(18283, (18283,))

In [126]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [127]:
X_final.shape, y_final.shape

((18283, 20), (18283,))

## Model Training

In [128]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [129]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 43ms/step - loss: 0.3253 - accuracy: 0.8472 - val_loss: 0.2000 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1416 - accuracy: 0.9450 - val_loss: 0.1943 - val_accuracy: 0.9186
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0992 - accuracy: 0.9613 - val_loss: 0.2133 - val_accuracy: 0.9125
Epoch 4/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0741 - accuracy: 0.9741 - val_loss: 0.2451 - val_accuracy: 0.9188
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0529 - accuracy: 0.9820 - val_loss: 0.2735 - val_accuracy: 0.9191
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0336 - accuracy: 0.9891 - val_loss: 0.3564 - val_accuracy: 0.9219
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0206 - accuracy: 0.9931 - val_loss: 0.3608 - val_accuracy: 0.9074
E

## Adding Dropout

In [130]:
'''
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
'''

"\nfrom tensorflow.keras.layers import Dropout\n## Creating model\nembedding_vector_features=40\nmodel=Sequential()\nmodel.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))\nmodel.add(Dropout(0.3))\nmodel.add(LSTM(100))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(1,activation='sigmoid'))\nmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])\n"

## Performance Metrics and Accuracy

In [131]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [132]:
y_pred = np.where(y_pred > 0.6, 1,0) # AUC ROC curve to assign the limit for y_pred

In [133]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3157,  258],
       [ 265, 2354]])

In [134]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9133244945309911

In [135]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3415
           1       0.90      0.90      0.90      2619

    accuracy                           0.91      6034
   macro avg       0.91      0.91      0.91      6034
weighted avg       0.91      0.91      0.91      6034

